In [17]:
#importing the required packages
import requests
import asyncio
import aiohttp
url_list=[]
url_list_crawled=[]
ext_cnt=0
req_cnt=0

#running thr first crawl to get the number of listed restaurants
def urllist(lat, long):
    headers={'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36',
             'accept-encoding': 'gzip, deflate, br',
             'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
             'accept-language': 'en-US,en;q=0.9',
             'if-none-match': 'W/"18a70-2svTzwERwbNJVGpuh7Z+/vbtj8U"'}
    #lat = 12.8933808
    #long = 77.6609237
    r1= requests.get('https://www.swiggy.com/dapi/restaurants/list/v5?lat=' + str(lat) + '&lng=' + str(long) + '&offset=0&sortBy=RELEVANCE&pageType=SEE_ALL&page_type=DESKTOP_SEE_ALL_LISTING', headers= headers)
    r1_js = r1.json()
    global url_list, tot_rest_cnt
    url_list=[]
    tot_rest_cnt = r1_js['data']['totalSize']
    for off in range(0, tot_rest_cnt, 14):
        url_list.append('https://www.swiggy.com/dapi/restaurants/list/v5?lat=' + str(lat) + '&lng=' + str(long) + '&offset=' + str(off) +'&sortBy=RELEVANCE&pageType=SEE_ALL&page_type=DESKTOP_SEE_ALL_LISTING')
    return url_list

# First async function for requesting to the site
async def request_to_site(session, url):
    global req_cnt
    req_cnt = req_cnt + 1
    async with session.get(url) as resp:
        x=await resp.json()
        print(x)
        return x

#Function for extracing the fields from the response json
def extract_fields(r1_js):
    global ext_cnt
    ext_cnt = ext_cnt+1
    tot_rest_cnt = r1_js['data']['totalSize']
    columns=['name', 'rest_id', 'city', 'area', 'rating', 'cuisine', 'cost_for_two', 'del_time', 'max_del_time', 'min_del_time', 
             'loc', 'disc_meta', 'disc_type', 'disc_op_type', 'fee', 'menu_link', 'slug']
    rn = len(r1_js['data']['cards'])
    for i in range(rn):
        cardType = r1_js['data']['cards'][i]['cardType']
        if cardType == 'restaurant':    
            name = r1_js['data']['cards'][i]['data']['data']['name']
            rest_id = r1_js['data']['cards'][i]['data']['data']['id']
            city = r1_js['data']['cards'][i]['data']['data']['city']
            area = r1_js['data']['cards'][i]['data']['data']['area']
            rating = r1_js['data']['cards'][i]['data']['data']['avgRating']
            cuisine = r1_js['data']['cards'][i]['data']['data']['cuisines']
            cost_for_two = r1_js['data']['cards'][i]['data']['data']['costForTwo']/100
            del_time = r1_js['data']['cards'][i]['data']['data']['deliveryTime']
            min_del_time = r1_js['data']['cards'][i]['data']['data']['minDeliveryTime']
            max_del_time = r1_js['data']['cards'][i]['data']['data']['maxDeliveryTime']
            loc = r1_js['data']['cards'][i]['data']['data']['locality']
            #discount attributes
            try:
                disc_meta = r1_js['data']['cards'][i]['data']['data']['aggregatedDiscountInfo']['shortDescriptionList'][0]['meta']
                disc_type = r1_js['data']['cards'][i]['data']['data']['aggregatedDiscountInfo']['shortDescriptionList'][0]['discountType']
                disc_op_type = r1_js['data']['cards'][i]['data']['data']['aggregatedDiscountInfo']['shortDescriptionList'][0]['operationType']
            except:
                disc_meta = '' 
                disc_type = ''
                disc_op_type = '' 

            #fee details
            fee = r1_js['data']['cards'][i]['data']['data']['feeDetails']['totalFees']/100
            try:
                menu_link = r1_js['data']['cards'][i]['data']['data']['cta']['link']
            except:
                menu_link = ''
            slug = r1_js['data']['cards'][i]['data']['data']['slugs']['restaurant']

            l2= [name, rest_id, city, area, rating, cuisine, cost_for_two, del_time, max_del_time, min_del_time, loc, disc_meta,
                   disc_type, disc_op_type, fee, menu_link, slug]
#            if dict(zip(columns, l2)) not in rest_json:
#                rest_json.append(dict(zip(columns, l2)))
            rest_json.append(dict(zip(columns, l2)))

#Second async function for running the crawls
async def run_crawls(session, url):
    global url_list_crawled
    url_list_crawled.append(url)
    try:
        js=await request_to_site(session,url)
        print(js)
        extract_fields(js)
    except Exception as err:
        #print(err)
        pass

    
#final code to run
async def getrest(lat, long):
    print('Inside getrest function')
    print('calling urllist function')
    url_list = urllist(lat, long)
    print('urllist function completed')
    global rest_json
    rest_json=[]
    print('starting the aiohttp requests')
    async with aiohttp.ClientSession() as session:
        await asyncio.gather(*[run_crawls(session, url) for url in url_list])
    print('ran all the crawls')
    #print(rest_json)
    return rest_json


In [ ]:
lat = 12.8933808
lon = 77.6609237
#import getrestlist
loop = asyncio.get_event_loop()
#json = await getrest(lat, lon)
json = loop.run_until_complete(getrest(lat, lon))
print(json)

In [19]:
lat = 12.8933808
lon = 77.6609237
json = await getrest(lat, lon)

Inside getrest function
calling urllist function
urllist function completed
starting the aiohttp requests
{'statusCode': 0, 'data': {'cacheExpiryTime': 320, 'pages': 1, 'pageIndex': 0, 'scrubber': 0, 'configs': {'ribbons': {'PREORDER': {'type': 'PREORDER', 'text': 'Preorder', 'textColor': '#ffffff', 'imageId': 'sfefefefeegeg', 'topBackgroundColor': '#d53d4c', 'bottomBackgroundColor': '#af2330', 'priority': 3}, 'EXCLUSIVE': {'type': 'EXCLUSIVE', 'text': 'Exclusive', 'textColor': '#ffffff', 'imageId': 'sfefefefeegeg', 'topBackgroundColor': '#fa4a5b', 'bottomBackgroundColor': '#d12a3b', 'priority': 2}, 'NEW': {'type': 'NEW', 'text': 'Newly Added', 'textColor': '#ffffff', 'imageId': 'sfefefefeegeg', 'topBackgroundColor': '#d53d4c', 'bottomBackgroundColor': '#af2330', 'priority': 4}, 'REPEAT': {'type': 'REPEAT', 'text': 'Repeat', 'textColor': '#ffffff', 'imageId': 'sfefefefeegeg', 'topBackgroundColor': '#D53D4C', 'bottomBackgroundColor': '#B02331', 'priority': 6}, 'CLOUD': {'type': 'CLOUD',

{'statusCode': 0, 'data': {'cacheExpiryTime': 320, 'pages': 1, 'pageIndex': 0, 'scrubber': 0, 'configs': {'ribbons': {'PREORDER': {'type': 'PREORDER', 'text': 'Preorder', 'textColor': '#ffffff', 'imageId': 'sfefefefeegeg', 'topBackgroundColor': '#d53d4c', 'bottomBackgroundColor': '#af2330', 'priority': 3}, 'EXCLUSIVE': {'type': 'EXCLUSIVE', 'text': 'Exclusive', 'textColor': '#ffffff', 'imageId': 'sfefefefeegeg', 'topBackgroundColor': '#fa4a5b', 'bottomBackgroundColor': '#d12a3b', 'priority': 2}, 'NEW': {'type': 'NEW', 'text': 'Newly Added', 'textColor': '#ffffff', 'imageId': 'sfefefefeegeg', 'topBackgroundColor': '#d53d4c', 'bottomBackgroundColor': '#af2330', 'priority': 4}, 'REPEAT': {'type': 'REPEAT', 'text': 'Repeat', 'textColor': '#ffffff', 'imageId': 'sfefefefeegeg', 'topBackgroundColor': '#D53D4C', 'bottomBackgroundColor': '#B02331', 'priority': 6}, 'CLOUD': {'type': 'CLOUD', 'text': 'Daily Menus', 'textColor': '#ffffff', 'imageId': 'sfefefefeegeg', 'topBackgroundColor': '#fa4a5b

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




ran all the crawls


In [ ]:
len(json)

In [21]:
unique = { each['rest_id'] : each for each in json }.values()
print('Total number of restaurants: ', tot_rest_cnt)
print('The number of restaurants in the data: ', len(json))
print('The number of restaurants in the data: ', len(unique))
print('The total number of urls is: ', len(url_list))
print('The total number of urls crawled: ', len(url_list_crawled))
print('The number of times extract fields is called: ', ext_cnt)
print('The number of times request to site is called: ', req_cnt)

Total number of restaurants:  823
The number of restaurants in the data:  939
The number of restaurants in the data:  687
The total number of urls is:  59
The total number of urls crawled:  118
The number of times extract fields is called:  118
The number of times request to site is called:  118


In [9]:
import pandas as pd

In [20]:
pd.DataFrame(json).to_csv('C:\\Users\\hp\\Desktop\\My Projects\\Restaurants\\Outputs\\compare_test4.csv')

In [ ]:
json

In [ ]:
unique = { each['rest_id'] : each for each in json }.values()

In [ ]:
len(unique)

In [ ]:
list(range(0, tot_rest_cnt, 14))

In [ ]:
url_list[52:
        ]

In [ ]:
headers={'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36',
         'accept-encoding': 'gzip, deflate, br',
         'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
         'accept-language': 'en-US,en;q=0.9',
         'if-none-match': 'W/"18a70-2svTzwERwbNJVGpuh7Z+/vbtj8U"'}

In [ ]:
lat = 12.8933808
long = 77.6609237
r1= requests.get('https://www.swiggy.com/dapi/restaurants/list/v5?lat=' + str(lat) + '&lng=' + str(long) + '&offset=0&sortBy=RELEVANCE&pageType=SEE_ALL&page_type=DESKTOP_SEE_ALL_LISTING', headers= headers)
r1_js = r1.json()

In [ ]:
url_list=[]
tot_rest_cnt = r1_js['data']['totalSize']
for off in range(0, tot_rest_cnt, 14):
    url_list.append('https://www.swiggy.com/dapi/restaurants/list/v5?lat=' + str(lat) + '&lng=' + str(long) + '&offset=' + str(off) +'&sortBy=RELEVANCE&pageType=SEE_ALL&page_type=DESKTOP_SEE_ALL_LISTING')

In [ ]:
async def request_to_site(session, url):
    async with session.get(url) as resp:
        x=await resp.json()
        return x

In [ ]:
def extract_fields(r1_js):
    tot_rest_cnt = r1_js['data']['totalSize']
    columns=['name', 'rest_id', 'city', 'area', 'rating', 'cuisine', 'cost_for_two', 'del_time', 'max_del_time', 'min_del_time', 
             'loc', 'disc_meta', 'disc_type', 'disc_op_type', 'fee', 'menu_link', 'slug']
    rn = len(r1_js['data']['cards'])
    for i in range(rn):
        cardType = r1_js['data']['cards'][i]['cardType']
        if cardType == 'restaurant':    
            name = r1_js['data']['cards'][i]['data']['data']['name']
            rest_id = r1_js['data']['cards'][i]['data']['data']['id']
            city = r1_js['data']['cards'][i]['data']['data']['city']
            area = r1_js['data']['cards'][i]['data']['data']['area']
            rating = r1_js['data']['cards'][i]['data']['data']['avgRating']
            cuisine = r1_js['data']['cards'][i]['data']['data']['cuisines']
            cost_for_two = r1_js['data']['cards'][i]['data']['data']['costForTwo']/100
            del_time = r1_js['data']['cards'][i]['data']['data']['deliveryTime']
            min_del_time = r1_js['data']['cards'][i]['data']['data']['minDeliveryTime']
            max_del_time = r1_js['data']['cards'][i]['data']['data']['maxDeliveryTime']
            loc = r1_js['data']['cards'][i]['data']['data']['locality']
            #discount attributes
            try:
                disc_meta = r1_js['data']['cards'][i]['data']['data']['aggregatedDiscountInfo']['shortDescriptionList'][0]['meta']
                disc_type = r1_js['data']['cards'][i]['data']['data']['aggregatedDiscountInfo']['shortDescriptionList'][0]['discountType']
                disc_op_type = r1_js['data']['cards'][i]['data']['data']['aggregatedDiscountInfo']['shortDescriptionList'][0]['operationType']
            except:
                disc_meta = '' 
                disc_type = ''
                disc_op_type = '' 

            #fee details
            fee = r1_js['data']['cards'][i]['data']['data']['feeDetails']['totalFees']/100
            try:
                menu_link = r1_js['data']['cards'][i]['data']['data']['cta']['link']
            except:
                menu_link = ''
            slug = r1_js['data']['cards'][i]['data']['data']['slugs']['restaurant']

            l2= [name, rest_id, city, area, rating, cuisine, cost_for_two, del_time, max_del_time, min_del_time, loc, disc_meta,
                   disc_type, disc_op_type, fee, menu_link, slug]
            if dict(zip(columns, l2)) not in rest_json:
                rest_json.append(dict(zip(columns, l2)))

In [ ]:
async def run_crawls(session, url):
    try:
        js=await request_to_site(session,url)
        extract_fields(js)
    except Exception as err:
        #print(err)
        pass

In [ ]:
global rest_json
rest_json=[]
async with aiohttp.ClientSession() as session:
    await asyncio.gather(*[run_crawls(session, url) for url in url_list])

### Combining the fragments above and putting it into a single cell

In [ ]:
#importing the required packages
import requests
import asyncio
import aiohttp

#running thr first crawl to get the number of listed restaurants
def urllist(lat, long):
    headers={'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36',
             'accept-encoding': 'gzip, deflate, br',
             'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
             'accept-language': 'en-US,en;q=0.9',
             'if-none-match': 'W/"18a70-2svTzwERwbNJVGpuh7Z+/vbtj8U"'}
    #lat = 12.8933808
    #long = 77.6609237
    r1= requests.get('https://www.swiggy.com/dapi/restaurants/list/v5?lat=' + str(lat) + '&lng=' + str(long) + '&offset=0&sortBy=RELEVANCE&pageType=SEE_ALL&page_type=DESKTOP_SEE_ALL_LISTING', headers= headers)
    r1_js = r1.json()
    url_list=[]
    tot_rest_cnt = r1_js['data']['totalSize']
    for off in range(0, tot_rest_cnt, 14):
        url_list.append('https://www.swiggy.com/dapi/restaurants/list/v5?lat=' + str(lat) + '&lng=' + str(long) + '&offset=' + str(off) +'&sortBy=RELEVANCE&pageType=SEE_ALL&page_type=DESKTOP_SEE_ALL_LISTING')
    return url_list

# First async function for requesting to the site
async def request_to_site(session, url):
    async with session.get(url) as resp:
        x=await resp.json()
        return x

#Function for extracing the fields from the response json
def extract_fields(r1_js):
    tot_rest_cnt = r1_js['data']['totalSize']
    columns=['name', 'rest_id', 'city', 'area', 'rating', 'cuisine', 'cost_for_two', 'del_time', 'max_del_time', 'min_del_time', 
             'loc', 'disc_meta', 'disc_type', 'disc_op_type', 'fee', 'menu_link', 'slug']
    rn = len(r1_js['data']['cards'])
    for i in range(rn):
        cardType = r1_js['data']['cards'][i]['cardType']
        if cardType == 'restaurant':    
            name = r1_js['data']['cards'][i]['data']['data']['name']
            rest_id = r1_js['data']['cards'][i]['data']['data']['id']
            city = r1_js['data']['cards'][i]['data']['data']['city']
            area = r1_js['data']['cards'][i]['data']['data']['area']
            rating = r1_js['data']['cards'][i]['data']['data']['avgRating']
            cuisine = r1_js['data']['cards'][i]['data']['data']['cuisines']
            cost_for_two = r1_js['data']['cards'][i]['data']['data']['costForTwo']/100
            del_time = r1_js['data']['cards'][i]['data']['data']['deliveryTime']
            min_del_time = r1_js['data']['cards'][i]['data']['data']['minDeliveryTime']
            max_del_time = r1_js['data']['cards'][i]['data']['data']['maxDeliveryTime']
            loc = r1_js['data']['cards'][i]['data']['data']['locality']
            #discount attributes
            try:
                disc_meta = r1_js['data']['cards'][i]['data']['data']['aggregatedDiscountInfo']['shortDescriptionList'][0]['meta']
                disc_type = r1_js['data']['cards'][i]['data']['data']['aggregatedDiscountInfo']['shortDescriptionList'][0]['discountType']
                disc_op_type = r1_js['data']['cards'][i]['data']['data']['aggregatedDiscountInfo']['shortDescriptionList'][0]['operationType']
            except:
                disc_meta = '' 
                disc_type = ''
                disc_op_type = '' 

            #fee details
            fee = r1_js['data']['cards'][i]['data']['data']['feeDetails']['totalFees']/100
            try:
                menu_link = r1_js['data']['cards'][i]['data']['data']['cta']['link']
            except:
                menu_link = ''
            slug = r1_js['data']['cards'][i]['data']['data']['slugs']['restaurant']

            l2= [name, rest_id, city, area, rating, cuisine, cost_for_two, del_time, max_del_time, min_del_time, loc, disc_meta,
                   disc_type, disc_op_type, fee, menu_link, slug]
            if dict(zip(columns, l2)) not in rest_json:
                rest_json.append(dict(zip(columns, l2)))

#Second async function for running the crawls
async def run_crawls(session, url):
    try:
        js=await request_to_site(session,url)
        extract_fields(js)
    except Exception as err:
        #print(err)
        pass

    
#final code to run
async def getrest(lat, long):
    print('Inside getrest function')
    print('calling urllist function')
    url_list = urllist(lat, long)
    print('urllist function completed')
    global rest_json
    rest_json=[]
    print('starting the aiohttp requests')
    async with aiohttp.ClientSession() as session:
        await asyncio.gather(*[run_crawls(session, url) for url in url_list])
    print('ran all the crawls')
    print(rest_json)
    return rest_json


In [ ]:
import os
def restaurants():
    lat = 12.8933808
    lon = 77.6609237
    os.chdir('C:\\Users\\hp\\Desktop\\My Projects\\Restaurants\\website')
    import getrestlist
    loop = asyncio.get_event_loop()
    #json = await getrest(lat, lon)
    loop.run_until_complete(getrest(lat, lon))
    print(json)
    return json

In [ ]:
headers={'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36',
         'accept-encoding': 'gzip, deflate, br',
         'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
         'accept-language': 'en-US,en;q=0.9',
         'if-none-match': 'W/"18a70-2svTzwERwbNJVGpuh7Z+/vbtj8U"'}

In [ ]:
lat = 12.8933808
long = 77.6609237
r1= requests.get('https://www.swiggy.com/dapi/restaurants/list/v5?lat=' + str(lat) + '&lng=' + str(long) + '&offset=0&sortBy=RELEVANCE&pageType=SEE_ALL&page_type=DESKTOP_SEE_ALL_LISTING', headers= headers)
r1_js = r1.json()

In [ ]:
list(range(0, 400, 15))

In [ ]:

tot_rest_cnt = r1_js['data']['totalSize']
columns=['name', 'rest_id', 'city', 'area', 'rating', 'cuisine', 'cost_for_two', 'del_time', 'max_del_time', 'min_del_time', 
         'loc', 'disc_meta', 'disc_type', 'disc_op_type', 'fee', 'menu_link', 'slug']

for off in range(0, tot_rest_cnt, 14):
    r1=requests.get('https://www.swiggy.com/dapi/restaurants/list/v5?lat=' + str(lat) + '&lng=' + str(long) + '&offset=' + str(off) +'&sortBy=RELEVANCE&pageType=SEE_ALL&page_type=DESKTOP_SEE_ALL_LISTING', headers= headers)
    r1_js = r1.json()    
    try:
        rn = len(r1_js['data']['cards'])
    except:
        break
    for i in range(rn):
        cardType = r1_js['data']['cards'][i]['cardType']
        if cardType == 'restaurant':    
            name = r1_js['data']['cards'][i]['data']['data']['name']
            rest_id = r1_js['data']['cards'][i]['data']['data']['id']
            city = r1_js['data']['cards'][i]['data']['data']['city']
            area = r1_js['data']['cards'][i]['data']['data']['area']
            rating = r1_js['data']['cards'][i]['data']['data']['avgRating']
            cuisine = r1_js['data']['cards'][i]['data']['data']['cuisines']
            cost_for_two = r1_js['data']['cards'][i]['data']['data']['costForTwo']/100
            del_time = r1_js['data']['cards'][i]['data']['data']['deliveryTime']
            min_del_time = r1_js['data']['cards'][i]['data']['data']['minDeliveryTime']
            max_del_time = r1_js['data']['cards'][i]['data']['data']['maxDeliveryTime']
            loc = r1_js['data']['cards'][i]['data']['data']['locality']
            #discount attributes
            try:
                disc_meta = r1_js['data']['cards'][i]['data']['data']['aggregatedDiscountInfo']['shortDescriptionList'][0]['meta']
                disc_type = r1_js['data']['cards'][i]['data']['data']['aggregatedDiscountInfo']['shortDescriptionList'][0]['discountType']
                disc_op_type = r1_js['data']['cards'][i]['data']['data']['aggregatedDiscountInfo']['shortDescriptionList'][0]['operationType']
            except:
                disc_meta = '' 
                disc_type = ''
                disc_op_type = '' 

            #fee details
            fee = r1_js['data']['cards'][i]['data']['data']['feeDetails']['totalFees']/100
            try:
                menu_link = r1_js['data']['cards'][i]['data']['data']['cta']['link']
            except:
                menu_link = ''
            slug = r1_js['data']['cards'][i]['data']['data']['slugs']['restaurant']

            l2= [name, rest_id, city, area, rating, cuisine, cost_for_two, del_time, max_del_time, min_del_time, loc, disc_meta,
                   disc_type, disc_op_type, fee, menu_link, slug]
            if dict(zip(columns, l2)) not in rest_json:
                rest_json.append(dict(zip(columns, l2)))

In [ ]:
dict(zip(columns, l2)) not in rest_json

In [ ]:
set(rest_json)

In [ ]:
unique = { each['Name'] : each for each in rest_json}.values()

In [ ]:
import os
os.chdir('C:\\Users\\hp\\Desktop\\My Projects\\Restaurants\\website')

In [ ]:
import getrestlist

In [ ]:
import os
def restaurants():
    lat = 12.8933808
    lon = 77.6609237
    os.chdir('C:\\Users\\hp\\Desktop\\My Projects\\Restaurants\\website')
    import getrestlist
    loop = asyncio.get_event_loop()
    #json = await getrest(lat, lon)
    loop.run_until_complete(getrest(lat, lon))
    print(json)
    return json

In [ ]:
#loop = asyncio.get_event_loop()
js = restaurants()

In [ ]:
js

In [ ]:
rest_json

In [ ]:

loop.run_until_complete(coromain())  # run coromain() from sync code
pending = asyncio.Task.all_tasks()  # get all pending tasks
loop.run_until_complete(asyncio.gather(*pending)) 


In [ ]:
lat = 12.8933808
long = 77.6609237
json_ = await getrest(lat, long)

In [ ]:
import json
json.dumps(json_)